In [79]:
import numpy as np
import random
from typing import Callable
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [104]:
class Neuron:
    def __init__(self, act_func:Callable[[float], float]=None, input_neuron:bool=False):
        if input_neuron:
            self.weights = np.array([1])
            self.inp_neuron = input_neuron
            self.act_func = None
        else:
            self.weights = None
            self.inp_neuron = False
            self.act_func = act_func

    def compile(self, weights_sz:int):
        if self.act_func:
            self.weights = np.array([random.uniform(0, 10) for _ in range(weights_sz)])

    
    def forward(self, inp):
        # print(self.weights)
        # print(inp)
        z = self.weights.T @ inp
        if self.act_func:
            return self.act_func(z)
        else:
            return z

In [105]:
class Layer:
    def sigmoid(self, x:float)-> float:
        return 1/(1+np.exp(-x))
    
    def __init__(self, neurons:int, input_layer:bool=False, act_fctn:str=None, name:str=None):
        self.name = name
        if input_layer:
            self.inp_layer = input_layer
            self.neurons = [Neuron(act_fctn, True) for i in range(neurons)]
        else:
            if act_fctn == "sigmoid":
                self.inp_layer = input_layer               
                self.neurons = [Neuron(act_func=self.sigmoid) for i in range(neurons)]
            else:
                self.inp_layer = input_layer
                self.neurons = [Neuron() for i in range(neurons)]
    
    def __repr__(self):
        return self.name

    def compile(self, childs:int=None):
        print(f"{self}:{len(self.neurons)}")
        if not self.inp_layer:
            for i in self.neurons:
                i.compile(childs)
        return len(self.neurons)

    def fit(self, inp):
        out_list = []
        if self.inp_layer:
            for i in range(len(self.neurons)):
                out_list.append(self.neurons[i].forward(np.array([inp[0][i]])))
                print(out_list)
            return out_list
        else:
            for i in self.neurons:
                out_list.append(i.forward(inp))
            return out_list

In [106]:
class Tangrad:    
    def __init__(self, layer:list[Layer]=[]):
        self.layer = layer
    
    def add_layer(self, layer:Layer):
        self.layer.append(layer)
    
    def compile(self):
        compiled = 0
        for i in range(len(self.layer)):
            if i == 0:
                compiled = self.layer[i].compile()
            else:
                compiled = self.layer[i].compile(compiled)
        
    def fit(self, inp):
        prev_out = []
        for i in self.layer:
            if i == self.layer[0]:
                prev_out = i.fit(inp)
                print(f"output of {i} layer : {prev_out}")
            else:
                prev_out = np.array(i.fit(prev_out))
                print(f"output of {i} layer : {prev_out}")
        print(prev_out)

In [107]:
Ann = Tangrad()

In [108]:
layer_0 = Layer(4, True, None,"layer 0")
layer_1 = Layer(20, False, "sigmoid","layer 1")
layer_2 = Layer(10, False, "sigmoid","layer 2")
layer_3 = Layer(1, False, "sigmoid","layer 3")
Ann.add_layer(layer_0)
Ann.add_layer(layer_1)
Ann.add_layer(layer_2)
Ann.add_layer(layer_3)

In [109]:
Ann.compile()

layer 0:4
layer 1:20
layer 2:10
layer 3:1


In [67]:
iris  = load_iris()

In [29]:
x = iris.data[:, :]
y = iris.target_names[iris.target] == "virginica"
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=85)

In [31]:
scaler = StandardScaler()
traindata = scaler.fit_transform(x_train)
testdata = scaler.fit_transform(x_test)

In [111]:
Ann.fit(traindata)

[-1.0190456254199358]
[-1.0190456254199358, 0.7977480099732124]
[-1.0190456254199358, 0.7977480099732124, -1.2726164710416852]
[-1.0190456254199358, 0.7977480099732124, -1.2726164710416852, -1.0882512570775875]
output of layer 0 layer : [-1.0190456254199358, 0.7977480099732124, -1.2726164710416852, -1.0882512570775875]
output of layer 1 layer : [2.42792391e-07 7.98696172e-03 1.26491073e-08 2.85165048e-08
 6.58137754e-08 2.47375748e-05 1.49799015e-08 1.23807490e-05
 5.28378968e-05 7.01928590e-04 2.36117193e-12 6.84168109e-05
 9.55389958e-07 1.23741519e-07 1.34468086e-06 3.91581217e-06
 5.97583742e-10 7.19500341e-09 6.54924793e-05 3.01633147e-04]
output of layer 2 layer : [0.50906716 0.51393746 0.51032417 0.51017263 0.51402307 0.52059053
 0.50697937 0.50609161 0.50750165 0.50838353]
output of layer 3 layer : [1.]
[1.]
